In [89]:
import requests 
import pandas as pd 
import numpy as np
from dateutil import parser 

## Accéder à l'identifiant d'un film : cas pour un film 

In [2]:
key_api='30d362783462d26118823b9652e0df75' #clé de l'API 
film='The Fabelmans'

url=f"https://api.themoviedb.org/3/search/movie?api_key={key_api}&query={film}" 

#requête pour trouver les infos ayant le nom_du_film en title 
req = requests.get(url)
wb = req.json()

# on récupère tous les indexs des films en question 
index=[]
for film in range(len(wb['results'])):
    index.append(wb['results'][film]['id'])
index  

[804095, 1201609]

## Généralisation pour plusieurs films 

#### fonction pour recuperer l'id des films 

In [12]:
def get_id(nom):
    url_api=f"https://api.themoviedb.org/3/search/movie?api_key={key_api}&query={nom}" 
    req = requests.get(url_api)
    wb = req.json()

    index=[]
    for film in range(len(wb['results'])):
        index.append(wb['results'][film]['id'])
    return(index)

#### fonction pour récuperer les infos du film à partir de l'id 

In [43]:
def df_avec_id(id):
    id_film= id
    url_new_api = f"https://api.themoviedb.org/3/movie/{id_film}?api_key={key_api}&language=en-US"
    req_new = requests.get(url_new_api)
    wb_new = req_new.json()
    
    
    #ajustement des données 
    if 'belongs_to_collection' in wb_new and wb_new['belongs_to_collection'] is not None:
        wb_new['belongs_to_collection'] = wb_new['belongs_to_collection']['name']

        
    # Ajustement des données pour la clé 'genres'
    if 'genres' in wb_new:
        wb_new['genres'] = ', '.join([x['name'] for x in wb_new['genres']])
    else:
        wb_new['genres'] = None

# Ajustement des données pour la clé 'production_companies'
    if 'production_companies' in wb_new:
        wb_new['production_companies'] = ', '.join([x['name'] for x in wb_new['production_companies']])
    else:
        wb_new['production_companies'] = None

# Ajustement des données pour la clé 'production_countries'
    if 'production_countries' in wb_new:
        wb_new['production_countries'] = ', '.join([x['name'] for x in wb_new['production_countries']])
    else:
        wb_new['production_countries'] = None

# Ajustement des données pour la clé 'spoken_languages'
    if 'spoken_languages' in wb_new:
        wb_new['spoken_languages'] = ', '.join([x['name'] for x in wb_new['spoken_languages']])
    else:
        wb_new['spoken_languages'] = None

    
    df=pd.DataFrame(wb_new, index=[0])
    
    return (df)

#### Création de la database 

In [ ]:
def get_movie_info(movie_id_list):
    # Initialiser un DataFrame vide pour stocker les informations sur les films
    movie_df = pd.DataFrame()

    for movie_id in movie_id_list:
        # Utiliser la deuxième fonction pour obtenir les informations détaillées du film
        movie_info = df_avec_id(movie_id)

        # Vérifier si des informations ont été trouvées
        if movie_info is not None:
            # Ajouter les informations du film au DataFrame
            movie_df = pd.concat([movie_df, movie_info], ignore_index=True)

    return movie_df

#Exemple 
#movie_id_list = get_id("Inception") 
#result_df = get_movie_info(movie_id_list)
#result_df.head()

In [36]:
def create_movie_list(movie_list):
    all_movies_df = pd.DataFrame()

    for movie_name in movie_list:
        movie_id_list = get_id(movie_name)
        if movie_id_list:
            movie_info_df = get_movie_info(movie_id_list)
            # Ajouter les informations du film au DataFrame global
            all_movies_df = pd.concat([all_movies_df, movie_info_df], ignore_index=True)

    return all_movies_df

# Exemple 
#movie_list = ["Inception", "The Dark Knight", "Interstellar"]
#result_df = create_movie_list_dataframe(movie_list)
#result_df.head()

#### recuperer les titres des films sous forme de liste 

In [78]:
df_movies= pd.read_csv("df_all_data_allocine.csv")
liste_des_films = df_movies['Titre original'].tolist()
#print(liste_des_films)
#len(liste_des_films)

#### DATAFRAME 

In [44]:
%%time 
df_movie_info=create_movie_list(liste_des_films)

CPU times: user 21min 32s, sys: 1min 51s, total: 23min 24s
Wall time: 1h 45min 16s


In [48]:
len(df_movie_info)

42474

In [67]:
df_movie_info.head(20)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,success,status_code,status_message
0,False,/5N8LOeRzt7LcZIhPbNVaCsVOPdT.jpg,None,68000000.0,"Horror, Science Fiction, Mystery",https://www.nope.movie,762504.0,tt10954984,en,Nope,...,English,Released,What’s a bad miracle?,Nope,False,6.888,3400.0,NaN,NaN,NaN
1,False,None,None,50000.0,Comedy,,952663.0,tt15479832,en,NOPE!,...,,Released,"The short film is a reflection on our society,...",NOPE!,False,4.300,3.0,NaN,NaN,NaN
2,False,None,None,0.0,Comedy,,997143.0,None,de,Nein.,...,Deutsch,Released,,Nope.,False,0.000,0.0,NaN,NaN,NaN
3,False,None,None,0.0,,,574455.0,tt5948342,en,Pre-Nope,...,,Released,,Pre-Nope,False,0.000,0.0,NaN,NaN,NaN
4,False,None,None,0.0,Comedy,,658744.0,tt0133172,fi,Rakkaus on nopeampi Piiroisen pässiäkin,...,suomi,Released,,Rakkaus on nopeampi Piiroisen pässiäkin,False,0.000,0.0,NaN,NaN,NaN
5,False,None,None,300.0,"Comedy, Science Fiction",,1204691.0,tt29620717,en,THE NOPE BOOK,...,"English, 日本語",Released,,THE NOPE BOOK,False,0.000,0.0,NaN,NaN,NaN
6,False,/wwdzmWC1Ra9SqPBrtCyRNJ4oYUO.jpg,None,35000000.0,"Western, Action",,12106.0,tt0114214,en,The Quick and the Dead,...,"English, Español",Released,Think you're quick enough?,The Quick and the Dead,False,6.530,1573.0,NaN,NaN,NaN
7,False,/k63u1PAW8aL7hy1Rwtn0d0oy4rn.jpg,None,25000000.0,"Drama, Adventure, History",,9912.0,tt0412080,en,The World's Fastest Indian,...,"Français, English",Released,Based On One Hell Of A True Story,The World's Fastest Indian,False,7.615,652.0,NaN,NaN,NaN
8,False,/e3wzLTfhd3ZzBo7nxklm4kvVJNO.jpg,None,2000000.0,Drama,http://studio.art-pictures.ru/en/proekty/buy-m...,460036.0,tt6964554,ru,Купи меня,...,"العربية, English, Pусский",Released,"Sex, Love and Dreams of a Beautiful Life",Buy Me,False,5.379,29.0,NaN,NaN,NaN
9,False,/i2wfrVqTPcosi2rZCCSD8tuGfnA.jpg,None,500000.0,Western,,76868.0,tt0055318,en,Posse from Hell,...,English,Released,Together They Team Up As An Avenging Posse Fro...,Posse from Hell,False,5.900,24.0,NaN,NaN,NaN


#### Nous avons dans cette base des films que nous avons pas dans notre liste donc nous allons enlever de la base tous les films qui ne sont pas concernés 

In [162]:
df_movies= pd.read_csv("df_all_data_allocine.csv")


In [163]:
print(df_movies.dtypes)
#df_movies.isnull().sum()
#df_movies["date"].value_counts()

Titre original     object
Note_presse        object
Note_spectateur    object
id_film_ac          int64
Box office         object
budget             object
visa               object
date               object
duree              object
dtype: object


In [164]:
#modifier les types numériques 

df_movies['Box office'] = df_movies['Box office'].str.replace(' ', '').astype(float)
df_movies['Note_presse'] = df_movies['Note_presse'].str.replace(',', '.').astype(float)
df_movies['Note_spectateur'] = df_movies['Note_spectateur'].replace("--", np.nan)
df_movies['Note_spectateur'] = df_movies['Note_spectateur'].str.replace(',', '.').astype(float)

In [165]:
#Modifier la date

# Fonction pour mapper les mois français aux mois anglais
def french_to_english_month(month_french):
    months_mapping = {
        'janvier': 'January',
        'février': 'February',
        'mars': 'March',
        'avril': 'April',
        'mai': 'May',
        'juin': 'June',
        'juillet': 'July',
        'août': 'August',
        'septembre': 'September',
        'octobre': 'October',
        'novembre': 'November',
        'décembre': 'December'
    }
    return months_mapping.get(month_french.lower(), month_french)

# Appliquer la fonction pour mapper les mois
df_movies['date'] = df_movies['date'].apply(lambda x: ' '.join([french_to_english_month(word) for word in x.split()]))

# Utiliser dateutil.parser.parse pour convertir les dates en objets datetime
df_movies['date'] = df_movies['date'].apply(lambda x: parser.parse(x, dayfirst=True))

# Convertir la colonne 'date' en datetime en utilisant pd.to_datetime
df_movies['date'] = pd.to_datetime(df_movies['date'])


In [166]:
#heures en minutes 
# Fonction pour convertir la durée en minutes
def convert_duration(duration_str):
    if isinstance(duration_str, str):
        # Séparer les heures et les minutes
        hours, minutes = map(int, duration_str.replace('h', '').replace('min', '').split())
        
        # Calculer la durée totale en minutes
        total_minutes = hours * 60 + minutes
        
        return float(total_minutes)
    else:
        # Gérer le cas où la valeur est déjà un nombre
        return float(duration_str)


# Appliquer la fonction de conversion
df_movies['duree_minutes'] = df_movies['duree'].apply(convert_duration)



In [167]:
df_movies.head()

,Titre original,Note_presse,Note_spectateur,id_film_ac,Box office,budget,visa,date,duree,duree_minutes
0,Nope,4.1,3.1,263062,113793605.0,-,-,2022-08-10,2h 10min,130.0
1,Empire Of Light,3.6,4.0,291817,292523.0,-,158782,2023-03-01,1h 56min,116.0
2,Babylon,4.0,4.2,275675,13513117.0,-,157633,2023-01-18,3h 09min,189.0
3,The Menu,2.8,3.3,271676,37628507.0,-,-,2022-11-23,1h 48min,108.0
4,The Fabelmans,4.9,4.3,255726,12105220.0,-,158402,2023-02-22,2h 31min,151.0
